In [1]:
import json
import copy

attributes  = []
attributes_type = {}
x = []
y = []
max_statistics = {}
group_statistics = {}

discrete=['Medical_History_1','Medical_History_10','Medical_History_15','Medical_History_24','Medical_History_32']
continous=['Product_Info_4', 'Ins_Age', 'Ht', 'Wt', 'BMI', 'Employment_Info_1', 'Employment_Info_4', 
           'Employment_Info_6', 'Insurance_History_5', 'Family_Hist_2', 'Family_Hist_3', 'Family_Hist_4',
           'Family_Hist_5']

missing = 74122.387
lines=[]

with open('training.csv') as f:
    r = f.readline()
    attributes = r.replace('\n','').split(',')
    for line in f:
        l = line.replace('\n', '').split(',')
        for ind in range( len(l) ):
            if (attributes[ind] in discrete):
                if (l[ind] !=''):
                    if ( not attributes[ind] in max_statistics ):    
                        max_statistics[attributes[ind]] = -1.0
                    else:
                        if (float(l[ind]) > max_statistics[attributes[ind]]):
                            max_statistics[attributes[ind]]=float(l[ind])
            elif (attributes[ind] not in continous and ind!=0 and attributes[ind] != 'Response' and  attributes[ind].find('Medical_Keyword')==-1 and l[ind]!=''):
                if (attributes[ind] not in group_statistics):
                     group_statistics[attributes[ind]]=[l[ind]]
                else:
                     if (l[ind] not in group_statistics[attributes[ind]]):
                        group_statistics[attributes[ind]].append(l[ind])  
        lines.append(line)
    for line in lines:
        l = line.replace('\n', '').split(',')
        temp = []
        for ind in range( len(l) ):
            if ( attributes[ind] in continous ):
                if( l[ind] == ''):
                    temp.append(missing)
                else:
                    temp.append( float(l[ind]) )
            elif ( attributes[ind].find('Medical_Keyword')!=-1 ):
                #dummy variable
                if( l[ind]=='1' ): 
                    temp.append(1)
                else:
                    temp.append(0)
            elif (attributes[ind] in discrete):
                if (l[ind] ==''):
                    temp.append(missing)
                else:
                    temp.append(float(l[ind])/max_statistics[attributes[ind]])
            elif (ind !=0 and( attributes[ind] != 'Response' )): 
                #catgorical variable  
                if( l[ind] == '' ):
                    for i in range (len(group_statistics[attributes[ind]])):
                        temp.append(0)
                else:
                    for i in range (len(group_statistics[attributes[ind]])):
                        if (l[ind]==group_statistics[attributes[ind]][i]):
                            temp.append(1)
                        else:
                            temp.append(0)
        x.append( copy.deepcopy(temp) )
        y.append( int(l[-1]) )
      
testing = []        
with open('testing.csv') as f:
    r = f.readline()
    attributes = r.replace('\n','').split(',')
    for line in f:
        l = line.replace('\n', '').split(',')
        temp = []
        for ind in range( len(l) ):
            if ( attributes[ind] in continous ):
                if( l[ind] == ''):
                    temp.append(missing)
                else:
                    temp.append( float(l[ind]) )
            elif ( attributes[ind].find('Medical_Keyword')!=-1 ):
                if( l[ind]=='1' ): 
                    temp.append(1)
                else:
                    temp.append(0)
            elif (attributes[ind] in discrete):
                if (l[ind] =='' ):
                    temp.append(missing)
                else:
                    if (float(l[ind])<=max_statistics[attributes[ind]]):
                        temp.append(float(l[ind])/max_statistics[attributes[ind]])      
                    else:
                        temp.append(1)
            elif (ind !=0 and( attributes[ind] != 'Response' )): 
                if( l[ind] == '' or l[ind] not in group_statistics[attributes[ind]] ):
                    for i in range (len(group_statistics[attributes[ind]])):
                        temp.append(0)
                else:
                    for i in range (len(group_statistics[attributes[ind]])):
                        if (l[ind]==group_statistics[attributes[ind]][i]):
                            temp.append(1)
                        else:
                            temp.append(0)  
        testing.append( copy.deepcopy(temp) )   

with open('processed_xgb.json', 'w') as f:
    f.write( json.dumps( {'x':x, 'y':y } ) )

with open('test_data_xgb.json', 'w') as f:
    f.write( json.dumps( testing ) )

['10', '26', '15', '29', '4', '37', '31', '28', '30', '36', '8', '5', '11', '19', '20', '24', '9', '21', '2', '3', '17', '13', '1', '34', '33', '6', '23']
